In [37]:
import numpy as np
from itertools import product
np.random.seed(10)

In [38]:
def linear_data(n):
    x = np.linspace(-0.5, 10, n)
    X = np.vstack([np.ones(len(x)), x]).T

    y_true = 2 + 0.5 * x

    noise = np.random.normal(0.3, 2, size=n)

    y_obs = y_true + noise

    interval_width = 0.3 * x
    y_lower = y_obs - interval_width
    y_upper = y_obs + interval_width

    x_grid = np.linspace(0, 10, 5)

    return n, X, y_true, y_lower, y_upper, x_grid

### Dependency Problem
We use OLS because this is the problem the paper is targeted towards although this proof remains true for any matrix

In [ ]:
n, X, y_true, y_lower, y_upper, x_grid = linear_data(3)
M = np.linalg.inv(X.T @ X) @ X.T


# --- Build Zonotope
# Midpoint-radius form
y_hat = (y_lower + y_upper)/2
r = (y_upper - y_lower)/2

# Build generators
segments = [M[:, i] * r[i] for i in range(len(r))]
print(segments)
c = M @ y_hat
# --- 

# --- Naive interval arithmetic
b_lower = np.zeros(M.shape[0])
b_upper = np.zeros(M.shape[0])

for j in range(M.shape[0]):
    for i in range(M.shape[1]):
        vals = [M[j,i]*y_lower[i], M[j,i]*y_upper[i]]
        b_lower[j] += min(vals)
        b_upper[j] += max(vals)
    
# ---

# --- Naive and Zonotope Regression Lines
bands_zono = []
bands_naive = []

for xi in x_grid:
    x = np.array([1, xi])
    centre = x @ c
    radius = sum(abs(x @ segments[i]) for i in range(len(segments)))
    bands_zono.append([centre - radius, centre + radius])

    y_pred_lower = x @ b_lower
    y_pred_upper = x @ b_upper
    bands_naive.append([y_pred_lower, y_pred_upper])

bands_zono = np.array(bands_zono)
bands_naive = np.array(bands_naive)
# ---

# --- Vertex Enum. Regression Lines
vertices = list(product(*zip(y_lower, y_upper)))
B_vertices = np.array([M @ v for v in vertices])

bands_vertices = []
for xi in x_grid:
    x = np.array([1, xi])
    projections = B_vertices @ x
    bands_vertices.append([projections.min(), projections.max()])

bands_vertices = np.array(bands_vertices)
# ---

print("\nNaive band:")
print(bands_naive)

print("\nZonotope band:")
print(bands_zono)

print("\n Vertex Enum. Band")
print(bands_vertices)


print(np.allclose(bands_zono, bands_vertices))


AttributeError: 'list' object has no attribute 'shape'